In [34]:
"""
まずは
"""
import pandas as pd

dir = "../data/"

df = pd.read_csv(dir + "windy_value/date/value_bottom_val_date.csv", header=None)
print(df)

#dataディレクトリの下のディレクトリ
windy_value = ["date","max_wind_speed","swell","swell_spacing","time_hour","wave_height","wind_speed"]

#data/*/ディレクトリの中の*_top_*.csvファイル
value_top_data = ["value_top_val_date.csv","value_top_val_max_wind_speed.csv","value_top_val_swell.csv",
                  "value_top_val_swell_spacing.csv","value_top_val_time_hour.csv","value_top_val_wave_height.csv",
                 "value_top_val_wind_speed.csv"]

             0           1           2           3           4           5  \
0   2020-08-13  2020-08-14  2020-08-15  2020-08-16  2020-08-17  2020-08-18   
1   2020-09-01  2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06   
2   2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07   
3   2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08   
4   2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08  2020-09-09   
..         ...         ...         ...         ...         ...         ...   
64  2020-12-14  2020-12-15  2020-12-16  2020-12-17  2020-12-18  2020-12-19   
65  2020-12-15  2020-12-16  2020-12-17  2020-12-18  2020-12-19  2020-12-20   
66  2020-12-16  2020-12-17  2020-12-18  2020-12-19  2020-12-20  2020-12-21   
67  2020-12-17  2020-12-18  2020-12-19  2020-12-20  2020-12-21  2020-12-22   
68  2020-12-23  2020-12-24  2020-12-25  2020-12-26  2020-12-27  2020-12-28   

             6           7           8           9  
0   2020-0

In [2]:
    eng_name_dic = {
        "taketomi_route" : ["isigaki_dep", "taketomi_dep"],
        "kurosima_route" : ["isigaki_dep", "kurosima_dep"],
        "kohama_route" : ["isigaki_dep", "tkohama_dep"],
        "iriomote_uehara_route" : ["isigaki_dep", "tuehara_dep"],
        "hatoma_route" : ["isigaki_dep", "hatoma_dep"],
        "iriomote_ohara_route" : ["isigaki_dep", "ohara_dep"],
       "hateruma_route" : ["isigaki_dep", "hateruma_dep"]
                                               } 

In [3]:
import pathlib
import glob
import re
import pprint
import os

csv_file = [os.path.basename(p) for p in glob.glob(dir + '/route/taketomi_route/isigaki_dep/*', recursive=True)
     if os.path.isfile(p) and re.search(r'2020-[0|1][8|9|10|11|12]-.*',p)]
csv_file = sorted(csv_file)

#print(csv_file)

#教師データのcsvファイル名の格納
print(re.findall('(.*).csv', csv_file[0]))
date_data = re.findall('(.*).csv', csv_file[0])
# ['2020-08-01']

date_in = df[df[0].isin([date_data])]
print(date_in)

df_bool = df[0] == date_data[0]
print(df_bool.values.sum())
if df_bool.values.sum() == 0:
    #for文continu
    pass
else:
    #日付がある場合なら処理(windy_valueがあるということ)
    pass

['2020-08-02']
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Index: []
0


In [63]:
#windyの数値データのデータフレームをリスト格納

value_file_list = []
for i in windy_value:
    value_file = glob.glob(os.path.join(dir + 'windy_value/'+i+'/', '*'))
    #print(dir + 'windy_value/'+i)
    value_file = sorted(value_file)
    value_file_list.append(value_file)    #2次元配列
#value_file_list = sorted(value_file_list)
    
#print(value_file_list)

df_7 = []

for i in value_file_list:
    print(i)
    for l, value in enumerate(i):
        #print(l)
        df_7.append(pd.read_csv(value, header=None))
        #print(df)
            #df_7[0] = df



['../data/windy_value/date/value_bottom_val_date.csv', '../data/windy_value/date/value_top_val_date.csv']
['../data/windy_value/max_wind_speed/value_bottom_val_max_wind_speed.csv', '../data/windy_value/max_wind_speed/value_top_val_max_wind_speed.csv']
['../data/windy_value/swell/value_bottom_val_swell.csv', '../data/windy_value/swell/value_top_val_swell.csv']
['../data/windy_value/swell_spacing/value_bottom_val_swell_spacing.csv', '../data/windy_value/swell_spacing/value_top_val_swell_spacing.csv']
['../data/windy_value/time_hour/value_bottom_val_time_hour.csv', '../data/windy_value/time_hour/value_top_val_time_hour.csv']
['../data/windy_value/wave_height/value_bottom_val_wave_height.csv', '../data/windy_value/wave_height/value_top_val_wave_height.csv']
['../data/windy_value/wind_speed/value_bottom_val_wind_speed.csv', '../data/windy_value/wind_speed/value_top_val_wind_speed.csv']


In [77]:
for i, value in enumerate(df_7):
    #print(value)
    print(i)
    if i == 0:    #bottom
        df_top = value
    elif i == 1:    #top
        df_bottom = value
    elif i%2 == 0:
        #print(df_top)
        df_top = pd.concat([df_top, value], axis=1, join='inner')
        print(df_top)
        break
        

0
1
2
            0           1           2           3           4           5   \
0   2020-08-13  2020-08-14  2020-08-15  2020-08-16  2020-08-17  2020-08-18   
1   2020-09-01  2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06   
2   2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07   
3   2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08   
4   2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08  2020-09-09   
..         ...         ...         ...         ...         ...         ...   
64  2020-12-14  2020-12-15  2020-12-16  2020-12-17  2020-12-18  2020-12-19   
65  2020-12-15  2020-12-16  2020-12-17  2020-12-18  2020-12-19  2020-12-20   
66  2020-12-16  2020-12-17  2020-12-18  2020-12-19  2020-12-20  2020-12-21   
67  2020-12-17  2020-12-18  2020-12-19  2020-12-20  2020-12-21  2020-12-22   
68  2020-12-23  2020-12-24  2020-12-25  2020-12-26  2020-12-27  2020-12-28   

            6           7           8           9   ...  

In [90]:
#日付の計算を行うための
import pandas.tseries.offsets as offsets

dir_label = dir + 'route/taketomi_route/isigaki_dep/' +csv_file[12]
print(dir_label)
def label_df(dir_label) :
    """
    日付ごとの教師データをdfに変換
    """
    df = pd.read_csv(dir_label, header=None)
    df[2]= pd.to_datetime(df[2])
    print(df)
    for row in df.itertuples():
        print(row[3]+offsets.Day(-1))
        print(type(row[3]))
        date_str = str(row[3]+offsets.Day(-1))
        date_str = re.sub('(\d*-\d*-\d*) (\d*:\d*:\d*)', '\\1', date_str)
        print(date_str)
        df_top_i = df_top[df_top[0].isin([date_str])]
        print(df_top_i)
        break
        #print(row.point)
        #if row[0] < 
label_df(dir_label)

../data/route/taketomi_route/isigaki_dep/2020-08-14.csv
        0  1          2
0   07:30  0 2020-08-14
1   08:30  0 2020-08-14
2   09:30  0 2020-08-14
3   10:30  0 2020-08-14
4   11:30  0 2020-08-14
5   13:00  0 2020-08-14
6   14:00  0 2020-08-14
7   15:00  0 2020-08-14
8   16:00  0 2020-08-14
9   17:00  0 2020-08-14
10  18:00  0 2020-08-14
2020-08-13 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-08-13


ValueError: cannot reindex from a duplicate axis

In [83]:
df_top

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,...,11,13,12,11,11.0,14.0,15.0,11.0,10.0,11.0
1,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,...,15,15,11,7,9.0,12.0,9.0,8.0,10.0,10.0
2,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,...,13,12,14,11,12.0,12.0,14.0,15.0,12.0,13.0
3,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,...,12,12,15,15,9.0,9.0,9.0,14.0,13.0,9.0
4,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,...,9,11,12,12,9.0,11.0,11.0,10.0,9.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,...,34,32,27,24,22.0,20.0,17.0,13.0,14.0,13.0
65,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,...,27,27,23,21,17.0,19.0,18.0,20.0,22.0,26.0
66,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,...,23,20,17,16,11.0,36.0,33.0,32.0,27.0,24.0
67,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,...,32,36,34,28,22.0,18.0,17.0,14.0,16.0,14.0
